In [111]:
from os import listdir
from os.path import isfile, join
import requests
from math import log

In [112]:
data_path = 'dane'
files = [f for f in listdir(data_path) if isfile(join(data_path, f))]

In [113]:
#zad 1-5
bigrams = {}
url = 'http://localhost:9200'

polish_letters = 'aąbcćdeęfghijklłmnńoópqrsśtuvwyxzźż'
def only_polish_letters(word):
    if len(word) == 0:
        return False
    for l in word:
        if not l in polish_letters:
            return False
    return True

j = 0
for filename in files:
    file = None
    with open(data_path +'/' + filename, 'r', encoding = 'utf-8') as f:
        file = f.read().lower()
        text = requests.post(url, data = file.encode('utf-8'), headers={'Content-type': 'text/plain; charset=utf-8'}).text
        text = [l for l in text.split('\n') if l != '']
        lines =  [l.split('\t') for l in text]
        for i in range(3, len(lines), 2):
            gram1 = lines[i-2][1] + ":" + lines[i-2][2].split(":")[0]
            gram2 = lines[i][1] + ":" + lines[i][2].split(":")[0]
            
            if only_polish_letters(lines[i-2][1]) and only_polish_letters(lines[i][1]):
                bigram = (gram1, gram2)
                bigrams[bigram] = bigrams.get(bigram, 0) + 1
            
    if j%100 == 0:
        print(j)
    j = j+1

0
100
200
300
400
500
600
700
800
900
1000
1100


In [114]:
list(bigrams.items())[:100]

[(('ustawa:subst', 'z:prep'), 8676),
 (('z:prep', 'dzień:subst'), 11416),
 (('o:prep', 'zmiana:subst'), 1413),
 (('zmiana:subst', 'ustawa:subst'), 908),
 (('ustawa:subst', 'o:prep'), 1696),
 (('o:prep', 'podatek:subst'), 592),
 (('podatek:subst', 'od:prep'), 480),
 (('od:prep', 'towar:subst'), 275),
 (('towar:subst', 'i:conj'), 616),
 (('i:conj', 'usługa:subst'), 643),
 (('usługa:subst', 'oraz:conj'), 144),
 (('oraz:conj', 'o:prep'), 945),
 (('podatek:subst', 'akcyzowy:adj'), 174),
 (('akcyzowy:adj', 'artykuł:brev'), 17),
 (('w:prep', 'ustawa:subst'), 5297),
 (('i:conj', 'numer:brev'), 8464),
 (('wprowadzać:fin', 'się:qub'), 1853),
 (('się:qub', 'następować:pact'), 206),
 (('następować:pact', 'zmiana:subst'), 198),
 (('w:prep', 'artykuł:brev'), 32184),
 (('dotychczasowy:adj', 'treść:subst'), 265),
 (('treść:subst', 'otrzymywać:fin'), 37),
 (('otrzymywać:fin', 'oznaczyć:ger'), 18),
 (('oznaczyć:ger', 'ustęp:brev'), 61),
 (('dodawać:fin', 'się:qub'), 8236),
 (('się:qub', 'ustęp:brev'), 3

In [115]:
#zad 6
# constructing tree to speedup search
def construct_tree(bigram):
    A__tree = {}
    _B_tree = {}
    for bigram, freq in bigrams.items():
        t_first = bigram[0]
        t_sec = bigram[1]

        if t_first not in A__tree:    
            A__tree[t_first] = {t_sec:freq}
        if t_sec not in A__tree[t_first]:
            A__tree[t_first][t_sec] = freq
        if t_sec not in _B_tree:    
            _B_tree[t_sec] = {t_first:freq}
        if t_first not in _B_tree[t_sec]:
            _B_tree[t_sec][t_first] = freq

    return A__tree, _B_tree 

A__tree, _B_tree = construct_tree(bigrams)
total_bigrams_occurances = sum(bigrams.values())

In [116]:
def H(vals, N):
    res = 0
    for val in vals:
        val /= N
        if not val == 0:
            res += val*log(val)
    return res

def LLR_bigram(a, b, A__tree, _B_tree):
    k11 = A__tree[a][b]
    k12 = sum(_B_tree[b].values()) - k11
    k21 = sum(A__tree[a].values()) - k11
    k22 = total_bigrams_occurances - k11 - k12 - k21
    N = k11 + k12 + k21 + k22
    return 2 * N * (H([k11,k12,k21,k22], N) - H([k11+k12,k21+k22], N) - H([k11+k21,k12+k22], N))

LLR_bigrams  = {k: LLR_bigram(k[0], k[1], A__tree, _B_tree) for k in bigrams.keys()}

LLR_bigrams

{('ustawa:subst', 'z:prep'): 42090.81636072441,
 ('z:prep', 'dzień:subst'): 53961.222144527695,
 ('o:prep', 'zmiana:subst'): 4519.396057399937,
 ('zmiana:subst', 'ustawa:subst'): 4464.751669856453,
 ('ustawa:subst', 'o:prep'): 4766.150524783235,
 ('o:prep', 'podatek:subst'): 1648.8773559392912,
 ('podatek:subst', 'od:prep'): 2542.921413737564,
 ('od:prep', 'towar:subst'): 932.2469612852102,
 ('towar:subst', 'i:conj'): 1667.6495096654082,
 ('i:conj', 'usługa:subst'): 1877.9605040243848,
 ('usługa:subst', 'oraz:conj'): 343.4553827434632,
 ('oraz:conj', 'o:prep'): 828.1400909700498,
 ('podatek:subst', 'akcyzowy:adj'): 1544.5175959992496,
 ('akcyzowy:adj', 'artykuł:brev'): 3.9288374991355606,
 ('w:prep', 'ustawa:subst'): 7923.659456668789,
 ('i:conj', 'numer:brev'): 54505.420234099016,
 ('wprowadzać:fin', 'się:qub'): 14134.025993255833,
 ('się:qub', 'następować:pact'): 840.5982269266489,
 ('następować:pact', 'zmiana:subst'): 1576.1085838708684,
 ('w:prep', 'artykuł:brev'): 114502.561557523

In [117]:
#zad 7
partitions = {}

for k,v in bigrams.items():
    partition_key = (k[0].split(':')[1], k[1].split(':')[1])
    partitions[partition_key] = partitions.get(partition_key, 0) + v 

partitions = {k: v for k,v in sorted(partitions.items(), key=lambda item: item[1], reverse=True)}

In [118]:
#zad 8
list(partitions.items())[:10]

[(('prep', 'subst'), 326073),
 (('subst', 'subst'), 287383),
 (('subst', 'adj'), 274258),
 (('adj', 'subst'), 187831),
 (('subst', 'prep'), 172751),
 (('subst', 'conj'), 84701),
 (('conj', 'subst'), 84224),
 (('prep', 'adj'), 79458),
 (('ger', 'subst'), 77472),
 (('prep', 'brev'), 67230)]

In [119]:
#zad 9
partition_LLR_dict = {}
for k,v in LLR_bigrams.items():
    partition_key = (k[0].split(':')[1], k[1].split(':')[1])
    if partition_key in partition_LLR_dict:
        partition_LLR_dict[partition_key][k] = v
    else:
        partition_LLR_dict[partition_key] = {k:v}
        



In [120]:
#zad 9
for k,v in list(partition_LLR_dict.items())[:10]:
    print(k)
    print(list(v.items())[:5])

('subst', 'prep')
[(('ustawa:subst', 'z:prep'), 42090.81636072441), (('ustawa:subst', 'o:prep'), 4766.150524783235), (('podatek:subst', 'od:prep'), 2542.921413737564), (('wyrób:subst', 'z:prep'), 45.31581303712945), (('dzień:subst', 'od:prep'), 13920.456983164755)]
('prep', 'subst')
[(('z:prep', 'dzień:subst'), 53961.222144527695), (('o:prep', 'zmiana:subst'), 4519.396057399937), (('o:prep', 'podatek:subst'), 1648.8773559392912), (('od:prep', 'towar:subst'), 932.2469612852102), (('w:prep', 'ustawa:subst'), 7923.659456668789)]
('subst', 'subst')
[(('zmiana:subst', 'ustawa:subst'), 4464.751669856453), (('rada:subst', 'minister:subst'), 18312.153020973128), (('droga:subst', 'rozporządzenie:subst'), 54049.33230070161), (('wyjątek:subst', 'wyrób:subst'), 75.47687961897606), (('przepis:subst', 'ustawa:subst'), 7812.138220124038)]
('subst', 'conj')
[(('towar:subst', 'i:conj'), 1667.6495096654082), (('usługa:subst', 'oraz:conj'), 343.4553827434632), (('średnik:subst', 'i:conj'), 1087.249762093

#zad 10
#a
Znaleziono bigramy zawierające m.in przyimki, spójniki, skróty, bądź też z zwroty przymiotnik, rzeczownik.
#b
Interesujące są m.in kategorie ('subst', 'subst'), ('subst', 'adj'), gdyż obie nie zawierają przyimków/spójniki, 
zawierają dzięki temu więcej treści, niż składni gramtycznej (jak np. z dnia).
Dzięki temu dostałam wyrażenia odpowiadające m.in Radzie ministrów, podatku akcyzowego.
#c
LLR zwraca utarte zwroty, które bardzo często zawierają przyimki/spójniki. 
Najlepszy efekt dało wybranie niektórych z bardziej popularnych kategori morfosyntaktycznych i w ramach nich tych kategori,
wybór zwrotów o najlepszej mierze LLR.
Zatem połączenie obu metod.
#d
Użycie kategorii morfosyntaktycznych może być przydatne w ekstracji informacji 
(po dezambiguacji, mamy pewne informacje o treści zdania).
Innym zastosowaniem jest translator np. polski -> ang (chociażby przetłumaczenie słowa zamek w odpowiednim kontekście).